In [ ]:
import pandas as pd
import torch
import numpy as np
from analysis.core import Core, calculate_B_C, evaluate_quantum_results_with_uncertainties, build_histograms, \
    build_results
import vector

vector.register_awkward()
import awkward as ak

In [28]:
# file_tag = 'pretrain-ema-ds1p0'
file_tag = 'scratch-ema-ds1p0-lr_half'

raw_data = torch.load(f'data/{file_tag}.pt')

## Truth Analysis

- `t  -> W+  b,     W+  -> l+  v`
- `t~ -> W-  b~,    W-  -> l-  v~`

---

extra truth variables columns: `[pt, eta, phi, mass, energy, index, mother1, mother2]`

In [29]:
raw_truth = {
    key.replace('EXTRA/lhe/', ''): torch.cat([item[key] for item in raw_data]).numpy()
    for key in raw_data[0].keys()
    if key.startswith('EXTRA/lhe/')
}


def sanity_and_merge(pairs, data):
    merged = {}
    for a, b, new_key in pairs:
        valid_a = ~np.isnan(data[a]).all(axis=1)
        valid_b = ~np.isnan(data[b]).all(axis=1)
        if np.any(valid_a & valid_b):
            raise ValueError(f"Conflict: both {a} and {b} present in same event.")
        valid_a = valid_a[:, None]  # broadcast
        merged[new_key] = np.where(valid_a, data[a], data[b])
    return merged


# Define pairs to merge: (key_a, key_b, merged_key)
pairs_to_merge = [
    ('e+', 'mu+', 'l+'),
    ('e-', 'mu-', 'l-'),
    ('nu(e)', 'nu(mu)', 'v'),
    ('nu(e)~', 'nu(mu)~', 'v~'),
]

# Run merging with sanity check
truth_data = sanity_and_merge(pairs_to_merge, raw_truth)
for k in ['W+', 'W-', 'b', 'b~', 't', 't~']:
    truth_data[k] = raw_truth[k]

for k in truth_data.keys():
    truth_data[k] = vector.zip({
        'pt': truth_data[k][:, 0],
        'eta': truth_data[k][:, 1],
        'phi': truth_data[k][:, 2],
        'mass': truth_data[k][:, 3],
    })

truth_core = Core(
    main_particle_1=truth_data['t'],
    main_particle_2=truth_data['t~'],
    child1=truth_data['l+'],
    child2=truth_data['l-'],
)

truth_result = truth_core.analyze()
truth_result = truth_result.query('mass < 400')
truth_hists = build_histograms(truth_result)

result, result_up, result_down = calculate_B_C(truth_hists, kappas=(1.0, -1.0))
D = -(result['C_nn'] + result['C_rr'] + result['C_kk'])
final = evaluate_quantum_results_with_uncertainties(result, result_up, result_down)

## Recon Analysis

### point cloud structure

- `full_input_point_cloud`: (N, 18, 7) array of particles in the event
    - columns: `[energy, pt, eta, phi, btag, isLepton, charge]`
- `t1 > b1, l1` l1 is 11 and 13
- `t2 > b2, l2` l2 is -11 and -13

> From LHE, lepton sign is correct, which means `l+` is for e+ and muon+; while for assignment, `l+` is for e- and muon-

In [30]:
def classify_TT2L(point_cloud, assignment_target, event_selection=None):
    """
    Classify particles in a TT2L topology.

    Parameters:
        point_cloud: (N, num_particles, num_features)
        assignment_target: tuple/list with 2 index arrays (each (N, 2)) for the two groups

    Returns:
        Dict with b1, b2, l1, l2 reconstructions.
        :param event_selection:
    """

    idx = np.arange(point_cloud.shape[0])[:, None]

    # Two targets (e.g., top1 and top2)
    t1_target = assignment_target[0]  # (N, 2)
    t2_target = assignment_target[1]  # (N, 2)

    # Gather candidates
    t1_recon_tmp = point_cloud[idx, t1_target, :].numpy()  # (N, 2, F)
    t2_recon_tmp = point_cloud[idx, t2_target, :].numpy()

    N = t1_recon_tmp.shape[0]

    if event_selection is None:
        event_selection = np.ones(N, dtype=bool)

    def select_object(recon_tmp, mask_feature_idx, threshold=0.5):
        """
        For each event, select the candidate if feature > threshold.
        At most one is expected to be True. Return (N, F) with NaN for none.
        """
        mask = recon_tmp[:, :, mask_feature_idx] > threshold  # (N, 2)
        idx_first = np.argmax(mask, axis=1)  # (N,)
        has_true = np.any(mask, axis=1)  # (N,)
        valid = has_true & event_selection  # only keep events that pass both conditions

        result = recon_tmp[np.arange(N), idx_first]  # (N, F)
        result[~valid] = np.nan  # Set to NaN if no valid candidate or not selected

        result = vector.zip({
            'pt': np.expm1(result[:, 1]),
            'eta': result[:, 2],
            'phi': result[:, 3],
            # 'mass': result[:, 4],
            'energy': np.expm1(result[:, 0]),
            'charge': result[:, 6],
        })

        return result

    # B-jet: feature[4] > 0.5
    b1_recon = select_object(t1_recon_tmp, 4)
    b2_recon = select_object(t2_recon_tmp, 4)

    # Lepton: feature[5] > 0.5
    l1_recon = select_object(t1_recon_tmp, 5)
    l2_recon = select_object(t2_recon_tmp, 5)

    return {
        'b1_recon': b1_recon,
        'b2_recon': b2_recon,
        'l1_recon': l1_recon,
        'l2_recon': l2_recon,
    }


def zip_two_neutrinos(neutrino_dict):
    def to_numpy(x):
        return x.detach().cpu().numpy() if hasattr(x, 'detach') else np.asarray(x)

    log_pt = to_numpy(neutrino_dict['log_pt'])  # (N, 2)
    eta = to_numpy(neutrino_dict['eta'])  # (N, 2)
    phi = to_numpy(neutrino_dict['phi'])  # (N, 2)

    pt = np.expm1(log_pt)

    nu1 = vector.zip({
        "pt": pt[:, 0],
        "eta": eta[:, 0],
        "phi": phi[:, 0],
        "mass": np.zeros_like(pt[:, 0]),  # neutrinos are massless
    })

    nu2 = vector.zip({
        "pt": pt[:, 1],
        "eta": eta[:, 1],
        "phi": phi[:, 1],
        "mass": np.zeros_like(pt[:, 1]),  # neutrinos are massless
    })

    return nu1, nu2


def extract_batch_assignments(batch, classify_fn, process="TT2L"):
    pred = batch['assignment_prediction']
    target = batch['assignment_target']
    target_mask = batch['assignment_target_mask']

    process_match = {
        'num_lepton': batch['full_input_point_cloud'].sum(axis=1)[:, 5].numpy().astype(np.int32),
        'num_bjet': batch['full_input_point_cloud'].sum(axis=1)[:, 4].numpy().astype(np.int32),
        'total_charge': batch['full_input_point_cloud'].sum(axis=1)[:, 6].numpy().astype(np.int32),
    }

    common_selection = (
            (process_match['num_bjet'] == 2) &
            (process_match['num_lepton'] == 2) &
            (process_match['total_charge'] == 0)
    )

    target_list = target[process]
    pred_process = pred[process]['best_indices']
    mask_process = target_mask[process]

    process_match.update({
        **classify_fn(batch['full_input_point_cloud'], target_list, event_selection=common_selection),
    })

    nu1_pred, nu2_pred = zip_two_neutrinos(batch['neutrinos']['predict'])
    nu1_true, nu2_true = zip_two_neutrinos(batch['neutrinos']['target'])
    process_match.update({
        "nu1_recon": nu1_pred,
        "nu2_recon": nu2_pred,
        "nu1_truth": nu1_true,
        "nu2_truth": nu2_true,
    })

    for p_idx, (assignment_target, assignment_prediction, assignment_target_mask) in enumerate(
            zip(target_list, pred_process, mask_process)):
        assignment_target = assignment_target.numpy()
        assignment_prediction = assignment_prediction.numpy()
        assignment_target_mask = assignment_target_mask.numpy()

        # Matching: true if all particles in the group are correctly assigned
        matched = (assignment_target == assignment_prediction)
        matched = matched.all(axis=1)  # along particle axis

        process_match[f"{process}_{p_idx}"] = matched
        process_match[f"{process}_{p_idx}_mask"] = assignment_target_mask

    return process_match


dfs = []
for batch in raw_data:
    out = extract_batch_assignments(batch, classify_fn=classify_TT2L)
    dfs.append(out)

# Instead of pd.concat, build one big awkward.Array
recon_data = ak.zip({
    k: ak.concatenate([out[k] for out in dfs])
    for k in dfs[0].keys()
})

truth_particle = {
    'b1': truth_data['b~'],
    'b2': truth_data['b'],
    'l1': truth_data['l-'],
    'l2': truth_data['l+'],
    't1': truth_data['t~'],
    't2': truth_data['t'],
}

for p, v in truth_particle.items():
    recon_data = ak.with_field(recon_data, v, f'{p}_truth')

recon_data = ak.with_field(recon_data, recon_data.l1_recon + recon_data.nu1_recon + recon_data.b1_recon, f't1_recon')
recon_data = ak.with_field(recon_data, recon_data.l2_recon + recon_data.nu2_recon + recon_data.b2_recon, f't2_recon')

truth_result = Core(
    main_particle_1=recon_data.t2_truth,
    main_particle_2=recon_data.t1_truth,
    child1=recon_data.l2_truth,
    child2=recon_data.l1_truth,
).analyze()

recon_result = Core(
    main_particle_1=recon_data.t2_recon,
    main_particle_2=recon_data.t1_recon,
    child1=recon_data.l2_recon,
    child2=recon_data.l1_recon,
).analyze()

full = build_results(truth_result, recon_result)

In [31]:
base_recon_cut = (recon_data["num_bjet"] == 2) & (recon_data["num_lepton"] == 2) & (recon_data["total_charge"] == 0)
mask_all_true = (
        recon_data["TT2L_0"] &
        recon_data["TT2L_0_mask"] &
        recon_data["TT2L_1"] &
        recon_data["TT2L_1_mask"]
        & base_recon_cut
)

valid_events = (
        base_recon_cut &
        recon_data.TT2L_0_mask &
        recon_data.TT2L_1_mask
)

# Count how many events have all four True
count = ak.sum(mask_all_true)
print("Number of events with all four True:", count, "out of", ak.sum(valid_events), "percentage:",
      count / ak.sum(valid_events) * 100)

Number of events with all four True: 172681 out of 241986 percentage: 71.35991338341887


### Unfolding

In [32]:
from analysis.unfold import main

bin_nums = 20 + 1
bin_edges = {
    "m_tt": np.array([0, 400, 500, 800, np.inf]),
    "B_Ak": np.linspace(-1, 1, bin_nums),
    "B_An": np.linspace(-1, 1, bin_nums),
    "B_Ar": np.linspace(-1, 1, bin_nums),
    "B_Bk": np.linspace(-1, 1, bin_nums),
    "B_Bn": np.linspace(-1, 1, bin_nums),
    "B_Br": np.linspace(-1, 1, bin_nums),
    "C_kk": np.linspace(-1, 1, bin_nums),
    "C_kn": np.linspace(-1, 1, bin_nums),
    "C_kr": np.linspace(-1, 1, bin_nums),
    "C_nk": np.linspace(-1, 1, bin_nums),
    "C_nn": np.linspace(-1, 1, bin_nums),
    "C_nr": np.linspace(-1, 1, bin_nums),
    "C_rk": np.linspace(-1, 1, bin_nums),
    "C_rn": np.linspace(-1, 1, bin_nums),
    "C_rr": np.linspace(-1, 1, bin_nums),
}

full['weight'] = 19.29e3 * 140 / len(full)
unfolded = main(full, bin_edges=bin_edges, weight_col='weight')

# 1) Reshape all unfolded arrays for each variable (except m_tt)
mtt_nbins = len(bin_edges['m_tt']) - 1

unfolded_temp = {
    key: {
        'edges': edges,
        'counts': unfolded[f'{key}_recon_unfold_content'].to_numpy().reshape(mtt_nbins, len(edges) - 1),
        'errors': unfolded[f'{key}_recon_unfold_error'].to_numpy().reshape(mtt_nbins, len(edges) - 1),
    }
    for key, edges in bin_edges.items()
    if key != 'm_tt'
}

# 2) Split by m_tt bins using clean dict comprehension
unfolded_hists = {
    f"m_tt < {mtt_right}": {
        key: {
            'edges': data['edges'],
            'counts': data['counts'][idx],
            'errors': data['errors'][idx],
        }
        for key, data in unfolded_temp.items()
    }
    for idx, mtt_right in enumerate(bin_edges['m_tt'][1:])
}

result, result_up, result_down = calculate_B_C(unfolded_hists['m_tt < 400.0'], kappas=(1.0, -1.0))
D = -(result['C_nn'] + result['C_rr'] + result['C_kk'])
final = evaluate_quantum_results_with_uncertainties(result, result_up, result_down)

Category: recon - Variable: B_Ak - done
Category: recon - Variable: B_An - done
Category: recon - Variable: B_Ar - done
Category: recon - Variable: B_Bk - done
Category: recon - Variable: B_Bn - done
Category: recon - Variable: B_Br - done
Category: recon - Variable: C_kk - done
Category: recon - Variable: C_kn - done
Category: recon - Variable: C_kr - done
Category: recon - Variable: C_nk - done
Category: recon - Variable: C_nn - done
Category: recon - Variable: C_nr - done
Category: recon - Variable: C_rk - done
Category: recon - Variable: C_rn - done
Category: recon - Variable: C_rr - done


Info in <TCanvas::Print>: pdf file plots/unfolding/B_Ak_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/B_An_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/B_Ar_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/B_Bk_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/B_Bn_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/B_Br_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/C_kk_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/C_kn_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/C_kr_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/C_nk_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/C_nn_recon.pdf has been created
Info in <TCanvas::Print>: pdf file plots/unfolding/C_nr_recon.pdf has been created
Info

In [33]:
import pandas as pd
base_df = pd.DataFrame({
    'value': result,
    'uncertainty_up': result_up,
    'uncertainty_down': result_down
})

base_df['uncertainty_up'] = base_df['uncertainty_up'] - base_df['value']
base_df['uncertainty_down'] = base_df['value'] - base_df['uncertainty_down']

# Step 2: Combine with `final` entries (like 'Concurrence', 'Ckk + Cnn', etc.)
final_df = pd.DataFrame.from_dict(final, orient='index')
final_df.index.name = 'name'

# Step 3: Concatenate both
combined_df = pd.concat([base_df, final_df])

# Optional: Reset index if you prefer flat structure
combined_df.index.name = 'name'
combined_df = combined_df.reset_index()

# Save the combined DataFrame to a CSV file
combined_df.to_csv(f'{file_tag}.csv', index=False)

## Plotting

In [ ]:
from pathlib import Path
import os
from downstreams.plotting.unfolding import plot_uncertainty_with_ratio
from downstreams.plotting.kinematic_comparison import plot_kinematics_comparison
from matplotlib import pyplot as plt

bins_mtt = bin_edges["m_tt"]
mtt_labels = [
    r"$m_{t\bar{t}} < 400$",
    r"$400 < m_{t\bar{t}} < 500$",
    r"$500 < m_{t\bar{t}} < 800$",
    r"$m_{t\bar{t}} \geq 800$"
]

common_labels = {
    # 6 B terms: B_{A,B}{n,r,k}
    **{
        f"B_{which}{axis}": {
            "name": rf"$\cos\theta^{{{which}}}_{{{axis}}}$",
            "labels": [f"bin {i}" for i in range(len(bin_edges[f"B_{which}{axis}"]) - 1)],
        }
        for which in ['A', 'B']
        for axis in ['n', 'r', 'k']
    },
    # 9 C terms: C_{axis1}{axis2} for axis1, axis2 in {n,r,k}
    **{
        f"C_{ax1}{ax2}": {
            "name": rf"$\cos\theta^A_{{{ax1}}} \cos\theta^B_{{{ax2}}}$",
            "labels": [f"bin {i}" for i in range(len(bin_edges[f"C_{ax1}{ax2}"]) - 1)],
        }
        for ax1 in ['n', 'r', 'k']
        for ax2 in ['n', 'r', 'k']
    },
}

for var, var_cfg in common_labels.items():
    methods = [
        {
            "name": r"EveNet - Pretrain", "color": "green",
            # "data": unfolded[f"{var}_recon_unfold_error"]
            "data": unfolded[f"{var}_recon_unfold_content"]
        },
        {
            "name": r"EveNet - Scratch", "color": "green",
            "data": unfolded[f"{var}_recon_unfold_error"]
        },
    ]

    plot_uncertainty_with_ratio(
        mtt_labels, var_cfg["labels"], var_cfg['name'], methods,
        ratio_baseline_name=r"EveNet - Pretrain",
        p_dir=Path(os.getcwd()) / "plots",
        save_name=f"unfolded_{var}.pdf",
        ratio_baseline_max=0.25,
        ratio_baseline_min=-0.05,
        ratio_y_label=r"Improvement to EveNet - Pretrain",
    )



In [ ]:
p_dir = Path(os.getcwd()) / "plots"
named_configs = {
    "neutrino": {
        "variables": ["pt", "eta", "phi"],
        "x_labels": [r"$p_T^{\nu}$ [GeV]", r"$\eta^{\nu}$", r"$\phi^{\nu}$"],
        "kin_range": {"pt": (0, 350), "eta": (-np.pi * 1.5, np.pi * 1.5), "phi": (-np.pi, np.pi)},
        # "labels": [r"$\nu$ from $(top^+)$", r"$\nu$ from $(top^-)$"],
        "labels": [r"$\nu$ (scratch)", r"$\nu$ (pretrain)"],
        "colors": ['#5bb5ac', '#de526c'],
        "columns": ['nu'],
        "log_y": [True, False, False],
    },
    "top": {
        "variables": ["pt", "eta", "phi", "mass"],
        "x_labels": [r"$p_T^{t}$ [GeV]", r"$\eta^{t}$", r"$\phi^{t}$", r"$mass^{t}$ [GeV]"],
        "kin_range": {"pt": (0, 600), "eta": (-np.pi * 1.5, np.pi * 1.5), "phi": (-np.pi, np.pi), "mass": (100, 240)},
        # "labels": [r"$(top^+)$ ", r"$(top^-)$"],
        "labels": [r"$top$ (scratch)", r"$top$ (pretrain)"],
        "colors": ['#5bb5ac', '#de526c'],
        "columns": ['t'],
        "log_y": [True, False, False, False],
    },
    # "W": {
    #     "variables": ["pt", "eta", "phi", "mass"],
    #     "x_labels": [r"$p_T^{W}$ [GeV]", r"$\eta^{W}$", r"$\phi^{W}$", r"$mass^{W}$ [GeV]"],
    #     "kin_range": {"pt": (0, 350), "eta": (-np.pi * 1.5, np.pi * 1.5), "phi": (-np.pi, np.pi), "mass": (40, 120)},
    #     # "labels": [r"$(W^+)$", r"$(W^-)$"],
    #     "labels": [r"$W$ (scratch)", r"$W$ (pretrain)"],
    #     "colors": ['#5bb5ac', '#de526c'],
    #     "columns": ['W', 'plot_truth_W'],
    #     "log_y": [True, False, False, False],
    # },
    # "ttbar": {
    #     "variables": ["mass"],
    #     "x_labels": [r"$mass^{t\bar{t}}$ [GeV]"],
    #     "kin_range": {"mass": (350, 1000)},
    #     # "labels": [r"$(top^+)$ ", r"$(top^-)$"],
    #     "labels": [r"$t\bar{t}$ (scratch)", r"t\bar{t} (pretrain)"],
    #     "colors": ['#5bb5ac', '#de526c'],
    #     "columns": ['ttbar', 'plot_truth_ttbar'],
    #     "log_y": [False],
    # },
}

effective_recon_data = recon_data[
    base_recon_cut & (recon_data.l1_recon.pt > 0) & (recon_data.l2_recon.pt > 0) &
    (recon_data.b1_recon.pt > 0) & (recon_data.b2_recon.pt > 0)
    ]

for particle, cfg in named_configs.items():

    for i, var in enumerate(cfg["variables"]):
        fig, axs = plt.subplots(
            3, 1, figsize=(10, 16),
            gridspec_kw={'height_ratios': [3, 1, 2], 'hspace': 0.0},
            sharex=True
        )

        plot_kinematics_comparison(
            axs=axs,
            # kin=[getattr(nu[cfg['columns'][0]][..., 0], var), getattr(nu[cfg['columns'][0]][..., 1], var)],
            # truth_kin=[getattr(nu[cfg['columns'][1]][..., 0], var), getattr(nu[cfg['columns'][1]][..., 1], var)],
            kin=[
                ak.to_numpy(ak.concatenate([
                    getattr(effective_recon_data[f'{cfg["columns"][0]}1_recon'], var),
                    getattr(effective_recon_data[f'{cfg["columns"][0]}2_recon'], var)
                ], axis=0))
            ],
            truth_kin=[
                ak.to_numpy(ak.concatenate([
                    getattr(effective_recon_data[f'{cfg["columns"][0]}1_truth'], var),
                    getattr(effective_recon_data[f'{cfg["columns"][0]}2_truth'], var)
                ], axis=0))
            ],
            bins=100,
            kin_range=cfg["kin_range"][var],
            labels=cfg["labels"],
            colors=cfg["colors"],
            xlabel=cfg["x_labels"][i],
            normalize_col=cfg.get("normalize_col", False),
            log_z=cfg.get("log_z", True),
            log_y=cfg.get("log_y", [False, False, False, False])[i],
            c_percent=np.array([10, 100])
        )

        plt.tight_layout()
        if not os.path.exists(p_dir / "kinematics"):
            os.makedirs(p_dir / "kinematics")
        plt.savefig(p_dir / "kinematics" / f"{particle}_{var}.pdf")
        plt.close(fig)

In [ ]:
effective_recon_data = recon_data[base_recon_cut & (recon_data.l1_recon.pt > 0) & (recon_data.l2_recon.pt > 0)]

X = ak.to_numpy(ak.concatenate([
    getattr(effective_recon_data[f'b1_recon'], 'pt'),
    getattr(effective_recon_data[f'b2_recon'], 'pt')
], axis=0))

print("X has NaNs?", np.isnan(X).any())
print("X has inf?", np.isinf(X).any())
print("X type:", type(X))